# sub022

In [1]:
# import library
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns
from tqdm.notebook import tqdm
import pathlib
import plotly
import plotly.express as px
import itertools
import lightgbm as lgb
from optuna.integration import lightgbm as optuna_lgb
import simdkalman
import optuna
import pyproj
from pyproj import Proj, transform
from sklearn import metrics
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix, accuracy_score
pd.set_option('display.max_rows', 100)
from math import * 

In [2]:
g1_4 = 'g1-4_v017'
g5 = 'g5_v015'

In [20]:
for g in ['g1-4_v015']:
    train = train.append(pd.read_csv(f'../output/{g}/{g}_train.csv'))
    test = test.append(pd.read_csv(f'../output/{g}/{g}_test.csv'))

In [21]:
test

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,phone,heightAboveWgs84EllipsoidM,x,y,z,xp,xdiff,yp,ydiff,zp,zdiff,dist,xnew,ynew,znew
0,2020-05-15-US-MTV-1,Pixel4,1273608785432,37.416605,-122.082041,2020-05-15-US-MTV-1_Pixel4,63.5,-2.693952e+06,-4.297518e+06,3.854256e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-05-15-US-MTV-1,Pixel4,1273608786432,37.416605,-122.082041,2020-05-15-US-MTV-1_Pixel4,63.5,-2.693952e+06,-4.297518e+06,3.854256e+06,-2.693952e+06,0.0,-4.297518e+06,0.0,3.854256e+06,0.0,0.0,NaN,NaN,NaN
2,2020-05-15-US-MTV-1,Pixel4,1273608787432,37.416605,-122.082041,2020-05-15-US-MTV-1_Pixel4,63.5,-2.693952e+06,-4.297518e+06,3.854256e+06,-2.693952e+06,0.0,-4.297518e+06,0.0,3.854256e+06,0.0,0.0,NaN,NaN,NaN
3,2020-05-15-US-MTV-1,Pixel4,1273608788432,37.416605,-122.082041,2020-05-15-US-MTV-1_Pixel4,63.5,-2.693952e+06,-4.297518e+06,3.854256e+06,-2.693952e+06,0.0,-4.297518e+06,0.0,3.854256e+06,0.0,0.0,NaN,NaN,NaN
4,2020-05-15-US-MTV-1,Pixel4,1273608789432,37.416605,-122.082041,2020-05-15-US-MTV-1_Pixel4,63.5,-2.693952e+06,-4.297518e+06,3.854256e+06,-2.693952e+06,0.0,-4.297518e+06,0.0,3.854256e+06,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85117,2021-04-29-US-MTV-2,SamsungS20Ultra,1303772667000,37.395828,-122.102952,2021-04-29-US-MTV-2_SamsungS20Ultra,63.5,-2.696264e+06,-4.297722e+06,3.852424e+06,-2.696264e+06,0.0,-4.297722e+06,0.0,3.852424e+06,0.0,0.0,NaN,NaN,NaN
85118,2021-04-29-US-MTV-2,SamsungS20Ultra,1303772668000,37.395828,-122.102952,2021-04-29-US-MTV-2_SamsungS20Ultra,63.5,-2.696264e+06,-4.297722e+06,3.852424e+06,-2.696264e+06,0.0,-4.297722e+06,0.0,3.852424e+06,0.0,0.0,NaN,NaN,NaN
85119,2021-04-29-US-MTV-2,SamsungS20Ultra,1303772669000,37.395828,-122.102952,2021-04-29-US-MTV-2_SamsungS20Ultra,63.5,-2.696264e+06,-4.297722e+06,3.852424e+06,-2.696264e+06,0.0,-4.297722e+06,0.0,3.852424e+06,0.0,0.0,NaN,NaN,NaN
85120,2021-04-29-US-MTV-2,SamsungS20Ultra,1303772670000,37.395828,-122.102952,2021-04-29-US-MTV-2_SamsungS20Ultra,63.5,-2.696264e+06,-4.297722e+06,3.852424e+06,-2.696264e+06,0.0,-4.297722e+06,0.0,3.852424e+06,0.0,0.0,NaN,NaN,NaN


In [19]:
test

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,phone,heightAboveWgs84EllipsoidM,x,y,z,xp,xdiff,yp,ydiff,zp,zdiff,dist,xnew,ynew,znew
0,2020-05-15-US-MTV-1,Pixel4,1273608785432,37.416605,-122.082041,2020-05-15-US-MTV-1_Pixel4,63.5,-2.693952e+06,-4.297518e+06,3.854256e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-05-15-US-MTV-1,Pixel4,1273608786432,37.416605,-122.082041,2020-05-15-US-MTV-1_Pixel4,63.5,-2.693952e+06,-4.297518e+06,3.854256e+06,-2.693952e+06,0.0,-4.297518e+06,0.0,3.854256e+06,0.0,0.0,NaN,NaN,NaN
2,2020-05-15-US-MTV-1,Pixel4,1273608787432,37.416605,-122.082041,2020-05-15-US-MTV-1_Pixel4,63.5,-2.693952e+06,-4.297518e+06,3.854256e+06,-2.693952e+06,0.0,-4.297518e+06,0.0,3.854256e+06,0.0,0.0,NaN,NaN,NaN
3,2020-05-15-US-MTV-1,Pixel4,1273608788432,37.416605,-122.082041,2020-05-15-US-MTV-1_Pixel4,63.5,-2.693952e+06,-4.297518e+06,3.854256e+06,-2.693952e+06,0.0,-4.297518e+06,0.0,3.854256e+06,0.0,0.0,NaN,NaN,NaN
4,2020-05-15-US-MTV-1,Pixel4,1273608789432,37.416605,-122.082041,2020-05-15-US-MTV-1_Pixel4,63.5,-2.693952e+06,-4.297518e+06,3.854256e+06,-2.693952e+06,0.0,-4.297518e+06,0.0,3.854256e+06,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85117,2021-04-29-US-MTV-2,SamsungS20Ultra,1303772667000,37.395828,-122.102952,2021-04-29-US-MTV-2_SamsungS20Ultra,63.5,-2.696264e+06,-4.297722e+06,3.852424e+06,-2.696264e+06,0.0,-4.297722e+06,0.0,3.852424e+06,0.0,0.0,NaN,NaN,NaN
85118,2021-04-29-US-MTV-2,SamsungS20Ultra,1303772668000,37.395828,-122.102952,2021-04-29-US-MTV-2_SamsungS20Ultra,63.5,-2.696264e+06,-4.297722e+06,3.852424e+06,-2.696264e+06,0.0,-4.297722e+06,0.0,3.852424e+06,0.0,0.0,NaN,NaN,NaN
85119,2021-04-29-US-MTV-2,SamsungS20Ultra,1303772669000,37.395828,-122.102952,2021-04-29-US-MTV-2_SamsungS20Ultra,63.5,-2.696264e+06,-4.297722e+06,3.852424e+06,-2.696264e+06,0.0,-4.297722e+06,0.0,3.852424e+06,0.0,0.0,NaN,NaN,NaN
85120,2021-04-29-US-MTV-2,SamsungS20Ultra,1303772670000,37.395828,-122.102952,2021-04-29-US-MTV-2_SamsungS20Ultra,63.5,-2.696264e+06,-4.297722e+06,3.852424e+06,-2.696264e+06,0.0,-4.297722e+06,0.0,3.852424e+06,0.0,0.0,NaN,NaN,NaN


In [3]:
gs = [g1_4, g5]

In [4]:
import ipynb_path

def get_nb_name():
    nb_path = ipynb_path.get()
    nb_name = nb_path.rsplit('/',1)[1].replace('.ipynb','')
    return nb_name

In [5]:
# directory setting
nb_name = get_nb_name()
INPUT = '../input/google-smartphone-decimeter-challenge'
OUTPUT = '../output/' + nb_name
os.makedirs(OUTPUT, exist_ok=True)

# utils

In [6]:
def get_train_score(df, gt):
    gt = gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
    df = df.merge(gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
    # calc_distance_error
    df['err'] = calc_haversine(df['latDeg_gt'], df['lngDeg_gt'], df['latDeg'], df['lngDeg'])
    # calc_evaluate_score
    df['phone'] = df['collectionName'] + '_' + df['phoneName']
    res = df.groupby('phone')['err'].agg([percentile50, percentile95])
    res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2 
    score = res['p50_p90_mean'].mean()
    return score

In [7]:
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [8]:
def visualize_trafic(df, center, zoom=9):
    fig = px.scatter_mapbox(df,
                            
                            # Here, plotly gets, (x,y) coordinates
                            lat="latDeg",
                            lon="lngDeg",
                            
                            #Here, plotly detects color of series
                            color="phoneName",
                            labels="phoneName",
                            
                            zoom=zoom,
                            center=center,
                            height=600,
                            width=800)
    fig.update_layout(mapbox_style='stamen-terrain')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(title_text="GPS trafic")
    fig.show()
    
def visualize_collection(df, collection):
    target_df = df[df['collectionName']==collection].copy()
    lat_center = target_df['latDeg'].mean()
    lng_center = target_df['lngDeg'].mean()
    center = {"lat":lat_center, "lon":lng_center}
    
    visualize_trafic(target_df, center)
    
def vis(df, phone):
    gt = get_ground_truth()
    gt['phone'] = gt['collectionName'] + '_' + gt['phoneName']
    tmp1 = df[df['phone']==phone].copy()
    tmp2 = gt[gt['phone']==phone].copy()
    
    tmp1['phoneName'] = 'train'
    tmp2['phoneName'] = 'gt'
    
    tmp = pd.concat([tmp1, tmp2])
    tmp['collectionName'] = 'all'
    
    lat_center = tmp['latDeg'].mean()
    lng_center = tmp['lngDeg'].mean()
    center = {"lat":lat_center, "lon":lng_center}
        
    visualize_trafic(tmp, center)

In [9]:
# ground_truth
def get_ground_truth():
    p = pathlib.Path(INPUT)
    gt_files = list(p.glob('train/*/*/ground_truth.csv'))

    gts = []
    for gt_file in gt_files:
        gts.append(pd.read_csv(gt_file))
    ground_truth = pd.concat(gts)

    return ground_truth

In [10]:
def percentile50(x):
    return np.percentile(x, 50)
def percentile95(x):
    return np.percentile(x, 95)

In [11]:
class train_result:
    def __init__(self, df):
        self.df = df
        self.gt = get_ground_truth()
        self.bl = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
        
        self.gt = self.gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
        self.df = self.df.merge(self.gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
        self.df['phone'] = self.df['collectionName'] + '_' + self.df['phoneName']
        self.df['err'] =  calc_haversine(self.df['latDeg_gt'], self.df['lngDeg_gt'], self.df['latDeg'], self.df['lngDeg'])
        
        self.phone_res = self.calc_err('phone')
        self.clc_res = self.calc_err('collectionName')
        self.phonename_res = self.calc_err('phoneName')
        
    def calc_err(self, by):
        res = self.df.groupby(by)['err'].agg([percentile50, percentile95])
        res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2
        return res
    
    @property
    def score(self):
        return self.phone_res['p50_p90_mean'].mean()
    @property
    def raw_data(self):
        return self.df
    @property
    def err(self):
        return self.phone_res
    @property
    def collection_err(self):
        return self.clc_res
    @property
    def phonename_err(self):
        return self.phonename_res
    
    def viz_map(self, collection, show_gt=True, show_bl=True):
        tmp = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp2 = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg_gt', 'lngDeg_gt']]
        tmp2 = tmp2.rename(columns={'latDeg_gt':'latDeg', 'lngDeg_gt':'lngDeg'})
        tmp2['phoneName'] = tmp2['phoneName'] + '_GT'
        tmp3 = self.bl[self.bl['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp3['phoneName'] = tmp3['phoneName'] + '_BL'
        
        if show_gt:
            tmp = tmp.append(tmp2)
        if show_bl:
            tmp = tmp.append(tmp3)
        visualize_collection(tmp, collection)

In [12]:
def get_data():
    base_train = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
    base_test = pd.read_csv(INPUT + '/' + 'baseline_locations_test.csv')
    sample_sub = pd.read_csv(INPUT + '/' + 'sample_submission.csv')
    ground_truth = get_ground_truth()
    return base_train, base_test, sample_sub, ground_truth

In [13]:
sub = pd.read_csv(INPUT + '/' + 'sample_submission.csv')
train = pd.DataFrame()
test = pd.DataFrame()

for g in gs:
    train = train.append(pd.read_csv(f'../output/{g}/{g}_train.csv'))
    test = test.append(pd.read_csv(f'../output/{g}/{g}_test.csv'))
    
print(len(train))
print(len(test))

131342
91486


In [14]:
result = train_result(train)
print(result.score)
display(result.err)

2.649795702022601


,percentile50,percentile95,p50_p90_mean
phone,,,
2020-05-14-US-MTV-1_Pixel4,0.925197,1.643060,1.284129
2020-05-14-US-MTV-1_Pixel4XLModded,0.848629,1.614393,1.231511
2020-05-14-US-MTV-2_Pixel4,1.911589,2.890980,2.401285
2020-05-14-US-MTV-2_Pixel4XLModded,1.812957,3.351212,2.582084
2020-05-21-US-MTV-1_Pixel4,2.031051,3.567717,2.799384
2020-05-21-US-MTV-2_Pixel4,0.692131,2.512237,1.602184
2020-05-21-US-MTV-2_Pixel4XL,0.630281,2.249620,1.439950
2020-05-29-US-MTV-1_Pixel4,1.732820,2.599610,2.166215
2020-05-29-US-MTV-1_Pixel4XL,1.732447,2.595623,2.164035


In [15]:
del sub['latDeg']
del sub['lngDeg']

In [16]:
sub = sub.merge(test[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']], on=['phone', 'millisSinceGpsEpoch'], how='left')

In [17]:
sub.to_csv(f'{OUTPUT}/{nb_name}_sub.csv', index=False)